## Fine-tune AraElectra on Custom Dataset

This is a POC for Fine-tuning AraElectra or other available pre-trained models: https://huggingface.co/aubmindlab on a Custom Dataset.

In [1]:
from src.finetuningQA.data_split import train_dev_test_split, combine_json_files
import json
import datasets
import os

In [2]:
# AraElectra model local path. 
# You can download another pretrained models from here: https://huggingface.co/aubmindlab
model_path= "araelectra-base-discriminator/"
model_name = "aubmindlab/araelectra-base-discriminator"

## Combine multiple datasets


In [3]:
#combine_json_files(["dataset/question-answering-dataset/Arabic-SQuAD.json","dataset/question-answering-dataset/arcd-train.json"])

## Splitt the dataset into train/dev/test (if needed)

In [4]:
#train_dev_test_split("dataset/question-answering-dataset/Arabic-SQuAD.json")

## Pre-process the data
Here we will prepare the dataset in the format that the model expect.

The Dataset structure must be as follow:

'''

    {"data": 

        [ 
            {"paragraphs": 

                [
                    {"qas": 
            
                    [{"question": "ما هو برنامج مزايا", "id": 445985, 

                        "answers": ["text": "هو أحد المبادرات الجديدة التي تطرحها وزارة الإسكان والتخطيط العمراني بالتعاون مع القطاع الخاص لتوفير السكن الاجتماعي للمواطنين المدرجة أسمائهم على قوائم الانتظار"}, 
            
                    {"question": "ما هو اقصى حد لقيمة القسط الشهري المترتب على المواطن؟", "id": 445986, 
                
                        "answers": ["text": "25% من راتب المواطن كحد أقصى."]

                    "context": "نبذة عن برنامج "مزايا " برنامج "مزايا " هو أحد المبادرات الجديدة التي تطرحها وزارة الإسكان والتخطيط العمراني بالتعاون مع القطاع الخاص لتوفير السكن الاجتماعي للمواطنين المدرجة أسمائهم على قوائم الانتظار، وتقوم فكرة البرنامج على قيام المواطن الذي تنطبق عليه المعايير بشراء وحدة سكنية من خلال حصوله على تمويل من أحد البنوك المشاركة، على أن تقوم الحكومة بتوفير الدعم المالي للمواطن، والمتمثل في سداد الفارق بين قيمة القسط الفعلي لمبلغ التمويل المحدد من قبل البنك الممول، وقيمة القسط الشهري المستحق على المواطن والذي لا يتجاوز 25% من راتب المواطن كحد أقصى. ",
            "document_id": 862498"}
                ]
            }
        ]

    }

        
'''


In [24]:
# Training Dataset
!python src/finetuningQA/preprocessing.py \
  --input_file="dataset/question-answering-dataset/turk_combined_all.json" \
  --output_file="dataset/question-answering-dataset/turk_combined_all_preprocessed.json" \
  --model_name=$model_name


# Eval Dataset

!python src/finetuningQA/preprocessing.py \
  --input_file="dataset/question-answering-dataset/arcd-test.json" \
  --output_file="dataset/question-answering-dataset/arcd-test_preprocessed.json" \
  --model_name=$model_name



aubmindlab/araelectra-base-discriminator
 72%|█████████████████████████████▋           | 232/321 [00:15<00:03, 26.47it/s]WARNING:tensorflow:Could not find answer for question '555832005630' :
 'الصين ( بالصينية المبسطة : تشونغوا ؛ بالصينية التقليدية : ) المعروفة رسميا باسم جمهورية الصين الشعبية ( بالصينية المبسطة : تشونغهوا رنمين غونغهيغو ؛ بالصينية التقليدية : ) هي الدولة الأكثر سكانا في العالم حيث يقطنها أكثر من 1 . 338 مليار نسمة .' 
vs.
 '中国'
orig answer:
 '中国'
 'الصين ( بالصينية المبسطة : تشونغوا ؛ بالصينية التقليدية : ) المعروفة رسميا باسم جمهورية الصين الشعبية ( بالصينية المبسطة : تشونغهوا رنمين غونغهيغو ؛ بالصينية التقليدية : ) هي الدولة الأكثر سكانا في العالم حيث يقطنها أكثر من 1 . 338 مليار نسمة .' 
vs.
 '中华人民共和国'
orig answer:
 '中华人民共和国'
100%|█████████████████████████████████████████| 321/321 [00:15<00:00, 20.72it/s]
aubmindlab/araelectra-base-discriminator
100%|██████████████████████████████████████████| 78/78 [00:00<00:00, 260.02it/s]


In [ ]:
# Load the preprocessed dataset and print one sample to see how the preprocessed data looks like
with open("dataset/question-answering-dataset/turk_combined_all_preprocessed.json") as f:
 data = json.load(f)['data']

print(data[2])

## Custom Dataset Loader

In [26]:
%%writefile src/finetuningQA/custom_dataset_loader.py
from __future__ import absolute_import, division, print_function

import json
import logging

import datasets


_CITATION = """\
@inproceedings{mozannar-etal-2019-neural,
    title = {Neural {A}rabic Question Answering},
    author = {Mozannar, Hussein  and Maamary, Elie  and El Hajal, Karl  and Hajj, Hazem},
    booktitle = {Proceedings of the Fourth Arabic Natural Language Processing Workshop},
    month = {aug},
    year = {2019},
    address = {Florence, Italy},
    publisher = {Association for Computational Linguistics},
    url = {https://www.aclweb.org/anthology/W19-4612},
    doi = {10.18653/v1/W19-4612},
    pages = {108--118},
    abstract = {This paper tackles the problem of open domain factual Arabic question answering (QA) using Wikipedia as our knowledge source. This constrains the answer of any question to be a span of text in Wikipedia. Open domain QA for Arabic entails three challenges: annotated QA datasets in Arabic, large scale efficient information retrieval and machine reading comprehension. To deal with the lack of Arabic QA datasets we present the Arabic Reading Comprehension Dataset (ARCD) composed of 1,395 questions posed by crowdworkers on Wikipedia articles, and a machine translation of the Stanford Question Answering Dataset (Arabic-SQuAD). Our system for open domain question answering in Arabic (SOQAL) is based on two components: (1) a document retriever using a hierarchical TF-IDF approach and (2) a neural reading comprehension model using the pre-trained bi-directional transformer BERT. Our experiments on ARCD indicate the effectiveness of our approach with our BERT-based reader achieving a 61.3 F1 score, and our open domain system SOQAL achieving a 27.6 F1 score.}
}
"""

_DESCRIPTION = """\
 Custom Dataset Loader 
"""

_URL = "../../dataset/question-answering-dataset/"
_URLs = {
    "train": _URL + "turk_combined_all_preprocessed.json",
    "dev": _URL + "arcd-test_preprocessed.json",
}



class CustomDatasetAraElectraConfig(datasets.BuilderConfig):
    """BuilderConfig for Custom Dataset."""

    def __init__(self, **kwargs):
        """BuilderConfig for CustomDataset.

        Args:
          **kwargs: keyword arguments forwarded to super.
        """
        super(CustomDatasetAraElectraConfig, self).__init__(**kwargs)


class CustomDatasetAraElectra(datasets.GeneratorBasedBuilder):
    """CustomDataset"""

    BUILDER_CONFIGS = [
        CustomDatasetAraElectraConfig(
            name="plain_text",
            version=datasets.Version("1.0.0", ""),
            description="Plain text",
        )
    ]

    def _info(self):
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "title": datasets.Value("string"),
                    "context": datasets.Value("string"),
                    "question": datasets.Value("string"),
                    "answers": datasets.features.Sequence(
                        {"text": datasets.Value("string"), "answer_start": datasets.Value("int32")}
                    ),
                }
            ),
            # No default supervised_keys (as we have to pass both question
            # and context as input).
            supervised_keys=None,
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        urls_to_download = _URLs
        downloaded_files = dl_manager.download_and_extract(urls_to_download)

        return [
            datasets.SplitGenerator(name=datasets.Split.TRAIN, gen_kwargs={"filepath": downloaded_files["train"]}),
            datasets.SplitGenerator(name=datasets.Split.VALIDATION, gen_kwargs={"filepath": downloaded_files["dev"]}),
        ]

    def _generate_examples(self, filepath):
        """This function returns the examples in the raw (text) form."""
        logging.info("generating examples from = %s", filepath)
        with open(filepath, encoding="utf-8") as f:
            data_ = json.load(f)
            for article in data_["data"]:
                title = article.get("title", "").strip()
                for paragraph in article["paragraphs"]:
                    context = paragraph["context"].strip()
                    for qa in paragraph["qas"]:
                        question = qa["question"].strip()

                        id_ = qa["id"]

                        answer_starts = [answer["answer_start"] for answer in qa["answers"]]
                        answers = [answer["text"].strip() for answer in qa["answers"]]                     

                        # Features currently used are "context", "question", and "answers".
                        # Others are extracted here for the ease of future expansions.
                        yield id_, {
                            "title": title,
                            "context": context,
                            "question": question,
                            "id": id_,
                            "answers": {"answer_start": answer_starts, "text": answers},
                        }


Overwriting src/finetuningQA/custom_dataset_loader.py


In [27]:
squad = datasets.load_dataset("src/finetuningQA/custom_dataset_loader.py")
print(squad)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset custom_dataset_loader downloaded and prepared to /home/mohammed/.cache/huggingface/datasets/custom_dataset_loader/plain_text/1.0.0/3d570e1d36dad6068475d5aab1fcc90b73dd204edf020b0f330a206c94a2bc29. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 49065
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 702
    })
})


## Fine-tuning 

In [28]:
!python src/finetuningQA/run_qa.py \
  --model_name_or_path $model_path \
  --dataset_name src/finetuningQA/custom_dataset_loader.py \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 8 \
  --gradient_accumulation_steps 1 \
  --learning_rate 3e-5 \
  --num_train_epochs 5 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ./run \
  --n_best_size 20 \
  --evaluation_strategy epoch \
  --save_steps 10000 \
  --seed 666 \
  --overwrite_output_dir \
  --warmup_steps 500

10/09/2022 17:16:43 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
10/09/2022 17:16:43 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precisi

## Move the new pretrained model to the "arabertv2-QA-model/" to be able to run inference with it

In [8]:
os.replace("run/pytorch_model.bin","araelectra-QA-model/pytorch_model.bin")
os.replace("run/tokenizer.json","araelectra-QA-model/tokenizer.json")

## Run Inference

In [6]:
context_1 = (
  'يعد ملف الإسكان بمملكة البحرين من أبرز الملفات المعيشة'
  'التي تحظى بالأولوية لدى المواطن البحريني، وهو ما وعت إليه'
  'القيادة الرشيدة منذ عقود طويلة، وتحديداً منذ مطلع ستينيات القرن الماضي، عندما تم تأسيس لجنة'
  'الإسكان والتمليك عام 1962، والتي وضعت التشريع الأول المنظم لسياسة تقديم الخدمات'
  'الإسكانية للمواطنين، ليعقب ذلك بعام واحد إنشاء مشروع مدينة عيسى كأول مدينة'
  'إسكانية متكاملة في المملكة تلبي احتياجات المواطنين في الحصول على السكن.'
  
  )

question1 = "متى تم تأسيس لجنة الإسكان والتمليك"
question2 = "ما هي أول مدينة اسكانية تم تأسيسيها في البحرين"


context_2 = (
'المكتبة البرلمانية هي مكتبة متخصصة ومركز للمعلومات والدراسات والبحوث القانونية تتبع إدارة البحوث والدراسات بالأمانة  العامة المساعدة لشؤون العلاقات' 
'والإعلام والبحوث بمجلس الشورى، وقد أنشئت عام 2016م لخدمة أعضاء المجلس، بالإضافة إلى فريق العمل الخاص بالأعضاء والعاملين'
'بالمجلس والباحثين في المجال نفسه من خارج المجلس كي تمدهم بالمعلومات اللازمة، كما تؤدي دورًاً أرشيفيًاً لحفظ الوثائق الخاصة'
'بأعمال البرلمان وجلساته،  وهي عبارة  عن  قاعة كبيرة تضم زهاء 4000 عنوان من الكتب العامة والمتخصصة والمراجع'
'والدوريات المتخصصة والمصادر الإلكترونية، والبحوث. .الهيكل التنظيمي للمكتبة. ترتبط المكتبة البرلمانية'
'هيكليا بإدارة البحوث والدراسات التي تتبع الأمانة العامة المساعدة لشؤون العلاقات والإعلام والبحوث'
)

question3 = "ما هي المكتبة البرلمانية"



In [11]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from src.finetuningQA.preprocess import ArabertPreprocessor

model = AutoModelForQuestionAnswering.from_pretrained("araelectra-QA-model/")
tokenizer = AutoTokenizer.from_pretrained("araelectra-QA-model/")

arabert_prep = ArabertPreprocessor(model_name=model_name)

context_ = arabert_prep.preprocess(context_2)
question_ = arabert_prep.preprocess(question3)

qa_model = pipeline("question-answering", model=model, tokenizer=tokenizer)
print("prediction {}".format(qa_model(question = question_, context = context_)))


prediction {'score': 0.7516324520111084, 'start': 22, 'end': 78, 'answer': 'مكتبة متخصصة ومركز للمعلومات والدراسات والبحوث القانونية'}
